# *Introduction to few key NLP metrics*

In any scenario, where there is a need to judge how good a machine translation of a given source language is, or judge the performance of a classifier, we need to introduce metrics. Note that, in case of machine translation source and target language can be anything--a PDF, latex code corresponding to a PDF, any human language, etc. Hence these metrics are language-independent. We attempt to introduce few of these metrics here:

+ Levenshtein distance (Edit distance)
+ BLEU (BiLingual Evaluation Understudy)
+ ROUGE (Todo)
+ METEOR (Todo)
+ Precision
+ Accuracy
+ Recall
+ F1 Score

## Machine Translations related

### Levenshtein distance
 Levenshtein or Edit distance is a fundamental string metric. Given two strings $A$ and $B$, it computes minimum number of insertions, deletions or replacements needed to transform $A$ to $B$. It helps to compute how much different a machine translated string is from the source string. Edit distance has a convenient recursive formula:
 
 $$lev_{a,b}(i,j) = \begin{cases} 
          lev_{a,b}(i+1,j+1) & if \ a[i]=b[j], a,b\neq\phi \\
          1+min\{lev_{a,b}(i,j+1),lev_{a,b}(i+1,j),lev_{a,b}(i+1,j+1)\} & if \ a[i] \neq b[j], a,b\neq \phi \\
          |a[i:]| & if \ b=\phi \\
          |b[j:]| & if \ b\neq\phi, a=\phi
       \end{cases}$$
       
where <mark>$lev_{a,b}(i,j)$ is the levenshtein distance of the string $a$ starting from $i^{th}$ index w.r.t. string $b$ starting from $j^{th}$ index</mark>. 

*EXPLANATION*: In order to compute $lev_{a,b}(i,j)$:
+ If $a[i] = b[j]$, then it is quite clear that we recursively compute the Edit distance of the string $a[i+1:]$ w.r.t. $b[j+1:]
 $
+ If $a[i] \neq b[j]$, then either:
  + we add $b[j]$ in the current position of $a$, so we recursively compute the Edit distance of the string $a[i:]$ w.r.t. $b[j+1:]$ (as $b[j]$ has been matched but didn't exhaust any character of the original $a$) and add $1$ to result(for the $Add$ operation).
  + we replace $a[i]$ with $b[j]$, so we recursively compute the Edit distance of the string $a[i+1:]$ w.r.t. $b[j+1:]$ (since $b[j]$ has been matched, and exhausted one more character of $a$) and add $1$ to result(for the $Replace$ operation).
  + we delete $a[i]$ in hope that the rest of the $a$ string matches with rest of the $b$ string, so we recursively compute Edit distance of $a[i+1:]$ w.r.t. $b[j]$(since character of $a$ has been exhausted without matching $b[j]$)
 
The recursive formula, naturally leads to a bottom up dynamic programming approach, where we take a $(|a|+1)$x$(|b|+1)$ matrix $dp$ with rows indexed by $a$ and column by $b$ such that (we output $dp[0][0]=lev_{a,b}(0,0)$ at the end):
$$ dp[i][j] =\begin{cases}
lev_{a,b}(i,j) & if \ 0\leq i\leq|a|, 0\leq j\leq|b|\\
lev_{a,\phi}(i,j) & if \ j=|b| \\
leb_{\phi,b}(i,j) & if \ i=|a|\\
\end{cases}$$


In [2]:
# runs in O(len(str1)*len(str2))
def levenshtein( str1, str2):
    l1 = len(str1)
    l2 = len(str2)
    matrix = [[-1]*(l2+1) for i in range(0,l1+1)] # initialisation of the (len(str1)+1)x(len(str2)+1) matrix with -1(s)
    
    # base cases
    for i in range(l2+1):
        matrix[l1][i] = l2 - i # initialisation of the bottom row
    
    for j in range(l1+1):
        matrix[j][l2] = l1 - j # initialisation of the rightmost column
        
    # computation starts!
    for i in range(l1-1,-1,-1): # bottom-up fashion--- from second last row to top row, and in each row from second-last cell to starting cell
        for j in range(l2-1,-1,-1):
            if str1[i]==str2[j]:
                matrix[i][j] = matrix[i+1][j+1] # if the characters are equal, move both pointers
            else:
                matrix[i][j] = 1 + min(matrix[i+1][j],matrix[i+1][j+1], matrix[i][j+1]) # else, add 1 (pertaining to either insert, replace or delete) to the minimum of the levenshstein distances of the remaining pieces of the strings
            
    
    return matrix[0][0]



Examples:
+ levenshtein distance of *"abc"* w.r.t *"adc"* is $1$ : *"abc"*  $\xrightarrow[]{\text{replace "b" with "d"}}$ *"adc"*
+ levenshtein distance of *"horse"* w.r.t. *"ros"* is $3$ : *"horse"* $\xrightarrow[]{\text{replace "h" with "r"}}$ *"rorse"* $\xrightarrow[]{\text{delete "r"}}$ *"rose"* $\xrightarrow[]{\text{delete "e"}}$ *"ros"* 

In [4]:
print(levenshtein("abc", "adc"))
print(levenshtein("horse","ros"))

1
3


### BLEU metric



<mark>Definition 1 (*$n-gram$*)</mark>: Generally an $n-gram$ is a sequence of $n$ consecutive words, letters, syllables or base pairs (*for machine translation(MT) purposes they are generally words*) collected from a text corpus(dataset).

Ex: bigrams($2$-grams) of *"This article is on NLP"* are *"This article"*, *"article is"*, *"on NLP"*, etc. 

Given an Machine Translation of a source "sentence", and one or more "references", we need to able to say how close the translaton is w.r.t the references and clearly distinguish a good candidate from a bad one.

<mark>Definition 2(*modified $n-gram$ precision*)</mark>